# We start from the Amplitude Amplification Tutorial from pennylane

# The main formular from the paper: [arXiv:2504.02385](https://arxiv.org/abs/2504.02385)

We implement it using RY as a rotational operator and use jax, as the clean python cell explodes with memory.

	
$$
\begin{bmatrix}
P(e^{i H}) & * \\
* & *
\end{bmatrix}
=
\Bigg(
\prod_{j=1}^d RY\big(\theta_{d+j}\big)\, \begin{bmatrix}
0 & 0 \\
0 & e^{-i H}
\end{bmatrix}

\Bigg)
\Bigg(
\prod_{j=1}^{d} RY\big(\theta_j\big)\,  \begin{bmatrix}
e^{i H} & 0 \\
0 & 0
\end{bmatrix} 
\Bigg)
RY\big(\theta_0)
$$




In [106]:
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

# In this demo d = 500 is for speed, the result is far away from the exact solution, at about 10000 it might be close.

d = 2 # degree of the polynomial, which is more or less the number of steps to approximate the imagary time evolution

def P(x):
    return jnp.cos(x)**(d-2)

# for performance they are moved into qsp_matrix
def R(theta):
    return jnp.array([
        [jnp.cos(theta/2), -jnp.sin(theta/2)],
        [jnp.sin(theta/2),  jnp.cos(theta/2)]
    ], dtype=jnp.complex128)

def U_signal(x, which):
    if which == 0:
        return jnp.array([[jnp.exp(1j*x), 0.0],
                          [0.0, 1.0]], dtype=jnp.complex128)
    else:
        return jnp.array([[1.0, 0.0],
                          [0.0, jnp.exp(-1j*x)]], dtype=jnp.complex128)

from jax import lax

@jax.jit
def qsp_matrix(angles, x):
    d = (angles.shape[0] - 1) // 2

    theta0 = angles[0]
    theta1 = angles[1:1+d]
    theta2 = angles[1+d:1+2*d]

    M = jnp.array([
        [jnp.cos(theta0/2), -jnp.sin(theta0/2)],
        [jnp.sin(theta0/2),  jnp.cos(theta0/2)]
    ], dtype=jnp.complex128)

    def loop0(M, theta):
        M = jnp.array([
        [jnp.cos(theta/2), -jnp.sin(theta/2)],
        [jnp.sin(theta/2),  jnp.cos(theta/2)]
    ], dtype=jnp.complex128) @ jnp.array([[jnp.exp(1j*x), 0.0],
                          [0.0, 1.0]], dtype=jnp.complex128) @ M
        return M, None

    def loop1(M, theta):
        M = jnp.array([
        [jnp.cos(theta/2), -jnp.sin(theta/2)],
        [jnp.sin(theta/2),  jnp.cos(theta/2)]
    ], dtype=jnp.complex128) @ jnp.array([[1.0, 0.0],
                          [0.0, jnp.exp(-1j*x)]], dtype=jnp.complex128) @ M
        return M, None

    M, _ = lax.scan(loop0, M, theta1)
    M, _ = lax.scan(loop1, M, theta2)

    return M

xs = jnp.linspace(0.0, 1.5, d * 23 // 11)
@jax.jit
def loss(angles):
    def loss_x(x):
        M = qsp_matrix(angles, x)
        amp = M[0,0]
        return jnp.abs(amp - P(x))**2

    return jnp.sum(jax.vmap(loss_x)(xs))

loss_grad = jax.jit(jax.grad(loss))


import numpy as np
from scipy.optimize import minimize

init_angles = np.random.uniform(0, np.pi, 2*d+1)

try:
    opt_angles = np.load(f'opt_angles_d{d}.npy')
    print(f"Loaded angles from opt_angles_d{d}.npy")
    print(f"Shape: {opt_angles.shape}")
except FileNotFoundError:
    print("No saved angles found, starting optimization.")
    res = minimize(loss, init_angles, method="BFGS", options={"maxiter":500}, jac=loss_grad)
    opt_angles = res.x
    print(res)
print("Optimized angles:", opt_angles)



No saved angles found, starting optimization.
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 1.2644400921506254e-08
        x: [ 2.089e+00  3.163e+00  1.770e-01  3.166e+00  3.996e+00]
      nit: 29
      jac: [ 8.658e-07 -8.172e-06  7.458e-07  9.992e-06  9.125e-07]
 hess_inv: [[ 2.214e+03 -1.615e+03 ... -1.832e+03  1.072e+03]
            [-1.615e+03  1.424e+03 ...  1.614e+03 -7.748e+02]
            ...
            [-1.832e+03  1.614e+03 ...  1.831e+03 -8.793e+02]
            [ 1.072e+03 -7.748e+02 ... -8.793e+02  5.207e+02]]
     nfev: 32
     njev: 32
Optimized angles: [2.08936709 3.16266604 0.17698618 3.16564583 3.99574496]


In [107]:
loss(opt_angles), P(0.2), qsp_matrix(opt_angles, 0.2)[0,0]

(Array(1.26444009e-08, dtype=float64),
 Array(1., dtype=float64, weak_type=True),
 Array(0.99993294+1.74855277e-05j, dtype=complex128))

In [108]:
import pennylane as qml
import numpy as np
from scipy.sparse.linalg import eigsh

L_x = 1
L_y = 2
boundary_conditions = [False, False]

t = 1
UU = 8

# Parameters for Trotterization and filter alignment
optimal_x_for_filter = 0.0
n_steps = 1
order = 2

H_penny = qml.spin.fermi_hubbard('rectangle',[L_x, L_y], hopping=t, coulomb=UU, boundary_condition=boundary_conditions)

H_original = H_penny.sparse_matrix()

eig_vals_orig, eig_vecs_orig = eigsh(H_original, k=6, which='SA')
print(f"PennyLane Ground state energy: {np.min(eig_vals_orig)}")

H_matrix = H_original
eig_max_orig = eig_max = eigsh(H_matrix, k=1, which='LA', return_eigenvectors=False)[0]
eig_min = np.min(eig_vals_orig)
print(f"Max eigenvalue: {eig_max}")
print(f"Min eigenvalue: {eig_min}")

print("Shifting Hamiltonian...")
ops = H_penny.operands
coefs = [op.scalar for op in ops]
obs = [op.base for op in ops]
H_penny = qml.Hamiltonian(coefs, obs)

H_penny = (H_penny - qml.Identity(wires=H_penny.wires) * (eig_min - optimal_x_for_filter * (eig_max - eig_min))) / (eig_max - eig_min)
H_penny = qml.Hamiltonian(H_penny.coeffs, H_penny.ops)

H_matrix = H_penny.sparse_matrix()
eig_max_check = eigsh(H_matrix, k=1, which='LA', return_eigenvectors=False)[0]
eig_min_check = eigsh(H_matrix, k=1, which='SA', return_eigenvectors=False)[0]
print(f"Max eigenvalue: {eig_max_check}")
print(f"Min eigenvalue: {eig_min_check}")

ops = H_penny.operands
coefs = [op.scalar for op in ops]
print(f"Number of terms: {len(ops)}")
print(f"Coefficients: {coefs[:5]}...")  # Show first 5
print(f"Operators: {ops[:5]}...")  # Show first 5

obs = [op.base for op in ops]
grouped = qml.pauli.group_observables(obs, coefs, grouping_type='commuting')
print(f"Number of groups: {len(grouped)}")
print(f"Shape of first group: {len(grouped[0])}")
print(f"Shape of second group: {len(grouped[1])}")
for g in grouped[0]:
    print(len(g))
for g in grouped[1]:
    print(len(g))
    
obs_flat = [item for sublist in grouped[0] for item in sublist]
coeffs_flat = [item for sublist in grouped[1] for item in sublist]

grouped_H = qml.Hamiltonian(coeffs_flat, obs_flat)
eig_vals_grouped, _ = eigsh(grouped_H.sparse_matrix(), k=6, which='SA')
print(f"Check Ground state energy: {np.min(eig_vals_grouped)}")
print(f"shape of H {grouped_H.sparse_matrix().shape[0]}")

# Remap wires of grouped_H to make wire 0 free for control
def our_R(theta, wire):
    qml.RY(theta, wires=wire) # @ qml.RZ(np.pi, wires=wire)

wire_map = {i: i+1 for i in range(grouped_H.num_wires)}
grouped_H_new = qml.map_wires(grouped_H, wire_map)

def iterate(angles):
    d = len(angles)//2
    theta0 = angles[0]
    theta1 = angles[1:d+1]
    theta2 = angles[d+1:2*d+1]

    our_R(theta0, wire=0)
    
    # Erste Schleife: wirkt auf |0> Komponente
    for t in theta1:
        qml.ctrl(qml.TrotterProduct(grouped_H_new, time=1.0, n=n_steps, order=order, check_hermitian=True), control=0, control_values=0)
        our_R(t, wire=0)
        
    # Zweite Schleife: wirkt auf |1> Komponente
    for t in theta2:
        qml.ctrl(qml.TrotterProduct(grouped_H_new, time=-1.0, n=n_steps, order=order, check_hermitian=True), control=0, control_values=1)
        our_R(t, wire=0)

devtest = qml.device("lightning.qubit", wires=grouped_H.num_wires + 1)

@qml.qnode(devtest)
def my_circ_ctrl(angles=None):
    # Prepare some state
    for w in range(1, grouped_H.num_wires + 1):
        qml.Hadamard(w)
    iterate(angles)
    return qml.state()     

rr = my_circ_ctrl(opt_angles)

psi_0 = np.ones(2**grouped_H.num_wires) / np.sqrt(2**grouped_H.num_wires)
r_now = psi_0  # rr.reshape(-1,2)[:,0]
energy = np.vdot(r_now, H_penny.sparse_matrix() @ r_now).real / np.vdot(r_now, r_now).real
print(f"Expectation value of initial state: {energy}")
r_now = rr.reshape(2,-1)[0]
energy = np.vdot(r_now, H_penny.sparse_matrix() @ r_now).real / np.vdot(r_now, r_now).real
print(f"Expectation value of Hamiltonian: {energy}")
energy = np.vdot(r_now, H_original @ r_now).real / np.vdot(r_now, r_now).real
print(f"Expectation value of original Hamiltonian: {energy}")
print(f"Norm r_now: {np.vdot(r_now, r_now).real}, Norm rr: {np.vdot(rr, rr).real}")

PennyLane Ground state energy: -1.0000000000000007
Max eigenvalue: 15.999999999999993
Min eigenvalue: -1.0000000000000007
Shifting Hamiltonian...
Max eigenvalue: 1.0000000000000004
Min eigenvalue: 4.213129211170115e-17
Number of terms: 12
Coefficients: [np.float64(-0.029411764705882366), np.float64(-0.029411764705882366), np.float64(0.23529411764705893), np.float64(-0.029411764705882366), np.float64(-0.029411764705882366)]...
Operators: (-0.029411764705882366 * (Y(0) @ Z(1) @ Y(2)), -0.029411764705882366 * (X(0) @ Z(1) @ X(2)), 0.23529411764705893 * I([0, 1, 2, 3]), -0.029411764705882366 * (Y(1) @ Z(2) @ Y(3)), -0.029411764705882366 * (X(1) @ Z(2) @ X(3)))...
Number of groups: 2
Shape of first group: 2
Shape of second group: 2
6
6
6
6
Check Ground state energy: 9.02565617616651e-18
shape of H 16
Expectation value of initial state: 0.2941176470588237
Expectation value of Hamiltonian: 0.29413125262860634
Expectation value of original Hamiltonian: 4.000231294686305
Norm r_now: 0.999891184

## Here we do Amplitude Amplification to get the filtered state with high probability

In [109]:
dev = qml.device("lightning.qubit", wires=grouped_H_new.num_wires + 2)
@qml.prod
def U2(wires):
    for wire in wires:
        qml.Hadamard(wires=wire)
    iterate(opt_angles)
    
@qml.prod
def oracle():
    qml.FlipSign(0, wires=0)

@qml.qnode(dev)
def circuit_state(iters):
    U2(wires=range(1, grouped_H_new.num_wires + 1))
    # qml.AmplitudeAmplification(U = U2(wires=range(1, grouped_H_new.num_wires + 1)),
    #                            O = oracle(),
    #                            iters = iters,
    #                            fixed_point=True,
    #                            work_wire=grouped_H_new.num_wires + 1)

    return qml.state()

test_output = circuit_state(iters=10)

tt = test_output.reshape(2,-1,2)[0,:,:].sum(axis=1)
print(f"Energy test_output: {np.vdot(tt, H_original @ tt).real / np.vdot(tt, tt).real}, with probability {np.vdot(tt, tt).real}")

Energy test_output: 4.000231294686305, with probability 0.9998911845154953


## Here we measure the phi, which is the energy of the shifted hamiltonian

$$
\phi = arc(<\psi|e^{i H}|\psi>),
$$
which is done with TrotterProduct and hadamard test.

In [110]:
dev_with_measurement = qml.device("default.qubit", wires=H_penny.num_wires + 3)  # the lightning device does not support post selection in this way yet
@qml.qnode(dev_with_measurement)
def circuit_energy():
    qml.StatePrep(test_output, wires=range(H_penny.num_wires + 2))
    return qml.expval(grouped_H_new)

@qml.qnode(dev_with_measurement)
def hadamard_test(measure="X"):
    ancilla = H_penny.num_wires + 2
    system_wires = range(H_penny.num_wires + 2)
    U2(wires=range(1, grouped_H_new.num_wires + 1))
    
    qml.Hadamard(wires=ancilla)
    qml.measure(wires=0, postselect=0) # The good from iteration
    
    qml.ctrl(
        qml.TrotterProduct(grouped_H_new, time=1.0, n=2, order=order),
        control=ancilla
    )
    
    if measure == "X":
        # return qml.expval(qml.PauliX(ancilla))
        qml.Hadamard(wires=ancilla)
        
    else:
        # return qml.expval(qml.PauliY(ancilla))
        qml.adjoint(qml.S)(ancilla)
        qml.Hadamard(wires=ancilla)
    
    return qml.expval(qml.PauliZ(ancilla))
    
energy = circuit_energy()
print(f"Energy from PennyLane QNode: {energy} (no good postselection)")
print(f"Energy from sparse matrix: {np.vdot(tt, grouped_H.sparse_matrix() @ tt).real / np.vdot(tt, tt).real}")


Re = hadamard_test("X")
Im = hadamard_test("Y")
z = Re + 1j*Im
phi = np.angle(z)  #  arg(<ψ|U|ψ>)
print(f"Energy from Hadamard test: {phi} ({Re} + {1j*Im})")

Energy from PennyLane QNode: 0.29411764786286126 (no good postselection)
Energy from sparse matrix: 0.29413125262860634
Energy from Hadamard test: 0.2907465510050087 (0.9107906582805925 + 0.2725322449710165j)


In [111]:
dev_with_measurement = qml.device("default.qubit", wires=H_penny.num_wires + 3)  # the lightning device does not support post selection in this way yet
@qml.qnode(dev_with_measurement)
def circuit_energy():
    qml.StatePrep(test_output, wires=range(H_penny.num_wires + 2))
    return qml.expval(grouped_H_new)

@qml.qnode(dev_with_measurement)
def hadamard_test(measure="X"):
    ancilla = H_penny.num_wires + 2
    system_wires = range(H_penny.num_wires + 2)
    U2(wires=range(1, grouped_H_new.num_wires + 1))
    
    qml.Hadamard(wires=ancilla)
    # qml.measure(wires=0, postselect=0) # The good from iteration
    
    qml.ctrl(
        qml.TrotterProduct(grouped_H_new, time=1.0, n=2, order=order),
        control=ancilla
    )
    
    if measure == "X":
        # return qml.expval(qml.PauliX(ancilla))
        qml.Hadamard(wires=ancilla)
        
    else:
        # return qml.expval(qml.PauliY(ancilla))
        qml.adjoint(qml.S)(ancilla)
        qml.Hadamard(wires=ancilla)
    
    return qml.state()
    return qml.expval(qml.PauliZ(ancilla))
    
energy = circuit_energy()
print(f"Energy from PennyLane QNode: {energy} (no good postselection)")
print(f"Energy from sparse matrix: {np.vdot(tt, grouped_H.sparse_matrix() @ tt).real / np.vdot(tt, tt).real}")


Re = hadamard_test("X")
Im = hadamard_test("Y")


Energy from PennyLane QNode: 0.29411764786286126 (no good postselection)
Energy from sparse matrix: 0.29413125262860634


In [112]:
@qml.qnode(dev_with_measurement)
def hadamard_test(measure="X"):
    ancilla = H_penny.num_wires + 2
    system_wires = range(H_penny.num_wires + 2)
    U2(wires=range(1, grouped_H_new.num_wires + 1))
    
    qml.Hadamard(wires=ancilla)
    # qml.measure(wires=0, postselect=0) # The good from iteration
    
    qml.ctrl(
        qml.TrotterProduct(grouped_H_new, time=1.0, n=2, order=order),
        control=ancilla
    )
    
    if measure == "X":
        # return qml.expval(qml.PauliX(ancilla))
        qml.Hadamard(wires=ancilla)
        
    else:
        # return qml.expval(qml.PauliY(ancilla))
        qml.adjoint(qml.S)(ancilla)
        qml.Hadamard(wires=ancilla)
    
    return qml.state()

psi_state = hadamard_test('X')
psi_flat = np.moveaxis(psi_state.reshape(2,16,2,2)[0,:,:,:], -1, 0).reshape(2, -1)
Z = np.array([[1, 0],
              [0, -1]])
rho_anc = psi_flat @ psi_flat.conj().T
expZ = np.trace(rho_anc @ Z).real
Re = expZ
print(f"Re={Re}")
psi_state = hadamard_test('Y')
psi_flat = np.moveaxis(psi_state.reshape(2,16,2,2)[0,:,:,:], -1, 0).reshape(2, -1)
Z = np.array([[1, 0],
              [0, -1]])
rho_anc = psi_flat @ psi_flat.conj().T
expZ = np.trace(rho_anc @ Z).real
Im =expZ
print(f"Im={Im}")
z = Re + 1j*Im
phi = np.angle(z)  #  arg(<ψ|U|ψ>)
print(f"Energy from Hadamard test: {phi} ({Re} + {1j*Im})")

Re=0.9106915501538353
Im=0.2725025892427385
Energy from Hadamard test: 0.29074655100500857 (0.9106915501538353 + 0.2725025892427385j)


In [113]:
@qml.qnode(dev_with_measurement, shots=1000000)
def circuit_sample(measure="Y"):
    ancilla = H_penny.num_wires + 2
    system_wires = range(H_penny.num_wires + 2)
    U2(wires=range(1, grouped_H_new.num_wires + 1))
    
    qml.Hadamard(wires=ancilla)
    # qml.measure(wires=0, postselect=0) # The good from iteration
    
    qml.ctrl(
        qml.TrotterProduct(grouped_H_new, time=1.0, n=2, order=order),
        control=ancilla
    )
    
    if measure == "X":
        # return qml.expval(qml.PauliX(ancilla))
        qml.Hadamard(wires=ancilla)
        
    else:
        # return qml.expval(qml.PauliY(ancilla))
        qml.adjoint(qml.S)(ancilla)
        qml.Hadamard(wires=ancilla)
    
    return qml.sample(wires=[0,ancilla])

samples = circuit_sample("X")

# Postselect on wire 0 == 0
mask = samples[:, 0] == 0
post = samples[mask]

p_success = len(post) / len(samples)
print("Postselection probability:", p_success)

anc_bits = post[:, 1]          # ancilla bit
z_vals = 1 - 2 * anc_bits     # 0→+1, 1→−1

Re = z_vals.mean()
print(f"Re={Re}")

samples = circuit_sample("Y")

# Postselect on wire 0 == 0
mask = samples[:, 0] == 0
post = samples[mask]

p_success = len(post) / len(samples)
print("Postselection probability:", p_success)

anc_bits = post[:, 1]          # ancilla bit
z_vals = 1 - 2 * anc_bits     # 0→+1, 1→−1

Im = z_vals.mean()
print(f"Im={Im}")

z = Re + 1j*Im
phi = np.angle(z)  #  arg(<ψ|U|ψ>)
print(f"Energy from Hadamard test: {phi} ({Re} + {1j*Im})")



# if I only use Y measurement
samples = circuit_sample()

# Postselect on wire 0 == 0
mask = samples[:, 0] == 0
post = samples[mask]

p_success = len(post) / len(samples)
print("Postselection probability:", p_success)

anc_bits = post[:, 1]          # ancilla bit
z_vals = 1 - 2 * anc_bits     # 0→+1, 1→−1

Im = z_vals.mean()
print(f"Im={Im}")

Re = np.sqrt(1 - Im**2)
print(f"Re={Re}")

z = Re + 1j*Im
phi = np.angle(z)  #  arg(<ψ|U|ψ>)
print(f"Energy from Hadamard test only using Y measurement: {phi} ({Re} + {1j*Im})")

Postselection probability: 0.999895
Re=0.9107546292360698
Postselection probability: 0.999896
Im=0.2739244881467673
Energy from Hadamard test: 0.2921598630445163 (0.9107546292360698 + 0.2739244881467673j)
Postselection probability: 0.99988
Im=0.2721586590390847
Re=0.9622523911687865
Energy from Hadamard test only using Y measurement: 0.2756356618541562 (0.9622523911687865 + 0.2721586590390847j)


In [114]:
from pennylane import transforms

decomposed_circuit = transforms.decompose(
    circuit_sample,
    gate_set={"CNOT", "RX", "RY", "RZ", "H", "S", "T"}
)

specs = qml.specs(decomposed_circuit)()
print(specs['resources'])

num_wires: 6
num_gates: 5368
depth: 3623
shots: Shots(total=1000000)
gate_types:
{'Hadamard': 486, 'RY': 389, 'RX': 4, 'RZ': 1465, 'CNOT': 2064, 'T': 960}
gate_sizes:
{1: 3304, 2: 2064}


In [115]:
output = qml.to_openqasm(decomposed_circuit)()
from qiskit import QuantumCircuit
qc = QuantumCircuit.from_qasm_str(output)

In [116]:
from qiskit_ibm_runtime.fake_provider import FakeProviderForBackendV2

provider = FakeProviderForBackendV2()

# Alle Backends
all_backends = provider.backends()
for b in all_backends:
    conf = b.configuration()
    print(f"Name: {b.name}, Qubits: {conf.num_qubits}, Basisgates: {conf.basis_gates}")

Name: fake_algiers, Qubits: 27, Basisgates: ['cx', 'id', 'rz', 'sx', 'x']
Name: fake_almaden, Qubits: 20, Basisgates: ['id', 'u1', 'u2', 'u3', 'cx']
Name: fake_armonk, Qubits: 1, Basisgates: ['id', 'rz', 'sx', 'x']
Name: fake_athens, Qubits: 5, Basisgates: ['id', 'rz', 'sx', 'x', 'cx', 'reset']
Name: fake_auckland, Qubits: 27, Basisgates: ['cx', 'id', 'rz', 'sx', 'x']
Name: fake_belem, Qubits: 5, Basisgates: ['id', 'rz', 'sx', 'x', 'cx', 'reset']
Name: fake_boeblingen, Qubits: 20, Basisgates: ['id', 'u1', 'u2', 'u3', 'cx']
Name: fake_bogota, Qubits: 5, Basisgates: ['id', 'rz', 'sx', 'x', 'cx', 'reset']
Name: fake_brisbane, Qubits: 127, Basisgates: ['ecr', 'id', 'rz', 'sx', 'x']
Name: fake_brooklyn, Qubits: 65, Basisgates: ['id', 'rz', 'sx', 'x', 'cx', 'reset']
Name: fake_burlington, Qubits: 5, Basisgates: ['id', 'u1', 'u2', 'u3', 'cx']
Name: fake_cairo, Qubits: 27, Basisgates: ['cx', 'ecr', 'id', 'rz', 'sx', 'x']
Name: fake_cambridge, Qubits: 28, Basisgates: ['id', 'u1', 'u2', 'u3', 'c

In [117]:
from qiskit import transpile


backend = provider.backend("fake_boeblingen")

# Transpile your QC for this backend
qc_transpiled = transpile(qc, backend=backend, optimization_level=3)

print("Depth:", qc_transpiled.depth())
print("Gate counts:", qc_transpiled.count_ops())


Depth: 5035
Gate counts: OrderedDict({'cx': 3616, 'u1': 1283, 'u3': 820, 'u2': 629, 'measure': 6})


In [ ]:
from qiskit import transpile
from qiskit_aer import AerSimulator

# idealer Simulator
sim = AerSimulator()

# ggf. nochmal für Simulator transpilen
qc_sim = transpile(qc_transpiled, sim)

# Run
job = sim.run(qc_sim, shots=100000)
result = job.result()
counts = result.get_counts()

ancilla = 5   # dein ancilla-wire
shots = sum(counts.values())

num = 0.0
den = 0.0

for bitstring, c in counts.items():
    b0   = int(bitstring[-1])              # wire 0
    banc = int(bitstring[-(ancilla+1)])    # ancilla

    if b0 == 0:  # Postselection
        z = 1 - 2*banc                     # Z-Eigenwert
        num += c * z
        den += c

expectation = num / den
p_success = den / shots

print(f"Exception value: {expectation}, with postselection probability {p_success}")

Im = expectation

Re = np.sqrt(1 - Im**2)
print(f"Re={Re}")

z = Re + 1j*Im
phi = np.angle(z)  #  arg(<ψ|U|ψ>)
print(f"Energy from Hadamard test only using Y measurement: {phi} ({Re} + {1j*Im})")


Exception value: 0.16740575383880338, with postselection probability 0.498137
Re=0.9858880837000019
Energy from Hadamard test only using Y measurement: 0.16819769798327694 (0.9858880837000019 + 0.16740575383880338j)


In [119]:
from qiskit_ibm_runtime import QiskitRuntimeService
 
# Run every time you need the service
service = QiskitRuntimeService()

In [120]:
backend = service.least_busy(simulator=False, operational=True)


# Transpile your QC for this backend
qc_transpiled = transpile(qc, backend=backend, optimization_level=3)

print("Depth:", qc_transpiled.depth())
print("Gate counts:", qc_transpiled.count_ops())


Depth: 9525
Gate counts: OrderedDict({'sx': 7237, 'rz': 3925, 'cz': 3541, 'x': 202, 'measure': 6})


In [130]:
from qiskit_ibm_runtime import Sampler

sampler = Sampler(mode=backend)

job = sampler.run(
        [qc_transpiled],
        shots=100000
    )

In [145]:
result = job.result()[0].data.c
counts = result.get_counts()

ancilla = 5   # dein ancilla-wire
shots = sum(counts.values())

num = 0.0
den = 0.0

for bitstring, c in counts.items():
    b0   = int(bitstring[-1])              # wire 0
    banc = int(bitstring[-(ancilla+1)])    # ancilla

    if b0 == 0:  # Postselection
        z = 1 - 2*banc                     # Z-Eigenwert
        num += c * z
        den += c

expectation = num / den
p_success = den / shots

print(f"Exception value: {expectation}, with postselection probability {p_success}")

Im = expectation

Re = np.sqrt(1 - Im**2)
print(f"Re={Re}")

z = Re + 1j*Im
phi = np.angle(z)  #  arg(<ψ|U|ψ>)
print(f"Energy from Hadamard test only using Y measurement: {phi} ({Re} + {1j*Im})")


Exception value: -0.03548203916396476, with postselection probability 0.54591
Re=0.9993703141962778
Energy from Hadamard test only using Y measurement: -0.035489488552424404 (0.9993703141962778 + (-0-0.03548203916396476j))
